In [1]:

import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
df=pd.read_csv('./data/fer2013.csv')

# print(df.info())
# print(df["Usage"].value_counts())

# print(df.head())
X_train,y_train,X_test,y_test=[],[],[],[]
num_labels = 7

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

y_train=np_utils.to_categorical(y_train, num_classes=num_labels)
y_test=np_utils.to_categorical(y_test, num_classes=num_labels)
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)


X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [8]:
len(X_train)

28709

### Create CNN architecture

In [10]:

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4)
num_labels = 7 
batch_size = 32
epochs = 200
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))


model.add(Dense(num_labels, activation='softmax'))
#model.summary()

#gen = ImageDataGenerator()
#train_generator = gen.flow(X_train, y_train, batch_size=batch_size)
 
model.compile(loss='categorical_crossentropy'
              , optimizer=Adam()
              , metrics=['accuracy']
             )
#val_datagen = ImageDataGenerator(rescale=1./255)
#val_generator = val_datagen.flow(X_test, y_test, batch_size=batch_size)

#Training the model
#print(shape(X_train))
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True,
          callbacks=[early_stopping])

#model.fit_generator(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs
                   # , validation_data=val_generator, validation_steps= len(X_test) // batch_size)

Epoch 1/200
898/898 [==============================] - 267s 297ms/step - loss: 1.8612 - accuracy: 0.2639 - val_loss: 1.5796 - val_accuracy: 0.3948
Epoch 2/200
898/898 [==============================] - 274s 305ms/step - loss: 1.5640 - accuracy: 0.3903 - val_loss: 1.4184 - val_accuracy: 0.4469
Epoch 3/200
898/898 [==============================] - 308s 342ms/step - loss: 1.4375 - accuracy: 0.4391 - val_loss: 1.4174 - val_accuracy: 0.4419
Epoch 4/200
898/898 [==============================] - 298s 332ms/step - loss: 1.3641 - accuracy: 0.4744 - val_loss: 1.3297 - val_accuracy: 0.4817
Epoch 5/200
898/898 [==============================] - 413s 460ms/step - loss: 1.2954 - accuracy: 0.5054 - val_loss: 1.2659 - val_accuracy: 0.5121
Epoch 6/200
898/898 [==============================] - 427s 476ms/step - loss: 1.2442 - accuracy: 0.5247 - val_loss: 1.2252 - val_accuracy: 0.5302
Epoch 7/200
898/898 [==============================] - 373s 415ms/step - loss: 1.1925 - accuracy: 0.5422 - val_loss: 1

In [62]:

num_labels = 7 
batch_size = 32
epochs = 200

model = Sequential()
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 0)

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
model.add(BatchNormalization())

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(BatchNormalization())

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(Flatten())
 
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
 model.add(Dense(num_labels, activation='softmax'))

#model.summary()

#gen = ImageDataGenerator()
#train_generator = gen.flow(X_train, y_train, batch_size=batch_size)
 
model.compile(loss='categorical_crossentropy'
              , optimizer=Adam()
              , metrics=['accuracy']
             )
#val_datagen = ImageDataGenerator(rescale=1./255)
#val_generator = val_datagen.flow(X_test, y_test, batch_size=batch_size)

#Training the model
#print(shape(X_train))
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True,
          callbacks=[early_stopping])

#model.fit_generator(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs
                   # , validation_data=val_generator, validation_steps= len(X_test) // batch_size)




Epoch 1/200
898/898 [==============================] - 5s 6ms/step - loss: 1.5455 - accuracy: 0.3911 - val_loss: 1.3818 - val_accuracy: 0.4728
Epoch 2/200
898/898 [==============================] - 5s 6ms/step - loss: 1.3249 - accuracy: 0.4931 - val_loss: 1.2690 - val_accuracy: 0.5093
Epoch 3/200
898/898 [==============================] - 5s 6ms/step - loss: 1.2315 - accuracy: 0.5347 - val_loss: 1.2640 - val_accuracy: 0.5110
Epoch 4/200
898/898 [==============================] - 5s 5ms/step - loss: 1.1727 - accuracy: 0.5537 - val_loss: 1.2336 - val_accuracy: 0.5305
Epoch 5/200
898/898 [==============================] - 6s 7ms/step - loss: 1.1180 - accuracy: 0.5775 - val_loss: 1.1727 - val_accuracy: 0.5550
Epoch 6/200
898/898 [==============================] - 5s 6ms/step - loss: 1.0693 - accuracy: 0.5987 - val_loss: 1.1678 - val_accuracy: 0.5639
Epoch 7/200
898/898 [==============================] - 5s 6ms/step - loss: 1.0173 - accuracy: 0.6126 - val_loss: 1.1764 - val_accuracy: 0.5628

In [12]:
train_score = model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

Train loss: 0.7169216871261597
Train accuracy: 72.74722456932068
Test loss: 1.1973587274551392
Test accuracy: 57.06325173377991


In [13]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")